In [1]:
import sys
sys.path.append("../")
import common.helpers as declass
import pandas as pd

In [2]:
config = declass.config("../common/mylogin.cnf")

In [3]:
cables_connector = declass.connect_db('declassification_cables', config)

In [ ]:
cables_tables = declass.show_tables(cables_connector)
cables_tables

In [ ]:
concepts = declass.df(cables_connector, "concepts")
declass.df_sample(cables_connector, "concepts", 5)

In [ ]:
network_nodes = declass.df(cables_connector, "network_nodes")
network_nodes = network_nodes[network_nodes.isnull().any(axis=1)==False]
network_nodes = network_nodes[['node_a','node_b']]
# network_nodes.to_json("cables_network.json",orient="records")
# network_nodes

In [ ]:
a_nodes = sorted(list(set(network_nodes['node_a'].astype(str))))

In [ ]:
b_nodes = sorted(list(set(network_nodes['node_b'].astype(str))))

In [ ]:
# network_nodes.groupby(['node_a','node_b']).size().reset_index().rename(columns={0:'count'})

In [ ]:
tags = declass.df(cables_connector, "tags")
tags

In [ ]:
# tag_doc = declass.df(cables_connector, "tag_doc")
# tag_doc

In [ ]:
# persons = persons.merge(person_doc.groupby('author_id')['play_id'].apply(list).reset_index(), how='left')

In [4]:
declass.rowcount(cables_connector, "tag_doc")

8605814

In [6]:
declass.df_sample(cables_connector, "tag_doc",5)

,tag_id,doc_id
0,13,1973KHARTO02100
1,13,1973LENING00586
2,13,1973MOGADI01397
3,13,1973NASSAU01116
4,13,1973NICOSI02503
